<h2> Implementation of a Decision Tree from scratch. </h2>

In [7]:
import numpy as np
import pandas as pd
import kagglehub
kagglehub.dataset_download("raghupalem/bill_authentication")

Using Colab cache for faster access to the 'bill_authentication' dataset.


'/kaggle/input/bill_authentication'

In [11]:
path = "/kaggle/input/bill_authentication/bill_authentication.csv"
df = pd.read_csv(path)

In [62]:
class Node:
    def __init__(self):
        self.label = None
        self.split = None
        self.left_child = None
        self.right_child = None

    def set_left_child(self, child: "Node"):
        self.left_child = child

    def set_right_child(self, child: "Node"):
        self.right_child = child

    def set_label(self, split: tuple[str, float], label=None):
        self.split = split
        self.label = label


class DecisionTree:
    def __init__(self, depth):
        self.depth = depth
        self.root = Node()

    def entropy(self,series):
        p = series.value_counts(normalize=True)
        return -np.sum(p * np.log2(p))

    def build(self, u : "Node", X : pd.DataFrame, y : pd.DataFrame, current_depth = 0) -> None:
        if (current_depth == self.depth):
            u.set_label(None, y.value_counts().idxmax())
            return
        N = len(X)
        entropy = self.entropy(y)
        best_split = []
        best_information_gain = -1

        for feature in X.columns:
            S = sorted(set(X[feature].to_list()))
            for split_threshold in S:
                mask = X[feature] <= split_threshold
                left_X, left_y = X[mask], y[mask]
                right_X, right_y = X[~mask], y[~mask]


                entropy_L = self.entropy(left_y)
                entropy_R = self.entropy(right_y)

                entropy_avg = (entropy_L + entropy_R) / 2
                information_gain = entropy - entropy_avg
                if information_gain > best_information_gain:
                    best_information_gain = information_gain
                    best_split = [feature, split_threshold]

        mask = X[best_split[0]] <= best_split[1]
        left_X, left_y = X[mask], y[mask]
        right_X, right_y = X[~mask], y[~mask]
        if len(left_X) == 0 or len(right_X) == 0:
            u.set_label(None, y.value_counts().idxmax())
            return
        u.set_left_child(Node())
        u.set_right_child(Node())
        u.set_label((best_split[0], best_split[1]))
        self.build(u.left_child, left_X, left_y, current_depth + 1)
        self.build(u.right_child, right_X, right_y, current_depth + 1)

    def fit(self, X, y):
        self.build(self.root, X, y)
        return

    def dfs(self, X, u : "Node") -> object:
        if (u.split == None):
            return u.label
        if X[u.split[0]] <= u.split[1]:
            return self.dfs(X, u.left_child)
        else:
            return self.dfs(X, u.right_child)

    def predict(self, X):
        rt = self.root
        ans = []
        for i, X in X.iterrows():
            ans.append(self.dfs(X,rt))
        return pd.Series(ans)

tree = DecisionTree(3)
df.head(5)
X = df.drop('Class', axis = 1)
y = df['Class'].copy()
tree.fit(X,y)

In [65]:
preds = tree.predict(X)
print((preds == y).sum() / len(X))

0.7849854227405247


In [56]:
from sklearn.tree import DecisionTreeClassifier
tree2 = DecisionTreeClassifier(max_depth = 3)
tree2.fit(X,y)
preds = tree2.predict(X)
print((preds == y).sum() / len(X))

0.9387755102040817
